# Running / Weight Combo Visualization

## Goals:

* Display running logs as bubble chart
  * X-axis is Date
  * Y-axis value corresponding to Pace
  * Size corresponding to distance
  * Some coloration corresponding to Tracker app used
  * Special color for races?
  * Hover-over shows details
* Weight plotting as scatter plot
  * X-axis is Date 
  * Y-axis is Weight in pounds
  * Light, tiny points
* Polynomial fit to weight
  * Poly fits look good (see: Weight Gurus)
  * Play around with number of degrees

In [151]:
%matplotlib inline

In [152]:
import pandas as pd
import numpy as np

## 1. Run Data

In [153]:
run_df = pd.read_csv('run.csv', parse_dates=['Date'], index_col=0).reset_index(drop=True)
run_df = run_df.fillna('')

In [165]:
# Planning on highlighting the race on the visualization
run_df['Line_Color'] = "#C0C0C0"  # Silver
run_df.loc[[237, 127], 'Line_Color'] = "#39FF14"  # Neon Green

In [156]:
# I'm tying distance to size, but need the numbers to be a little larger...
run_df['Display_Size'] = run_df['Distance'] * 2

## 2. Weigh-in Data

In [157]:
weight_df = pd.read_csv('weight.csv', parse_dates=['Date'], index_col=0)
weight_df = weight_df[weight_df['Date'] > '2017-02-28']
weight_df = weight_df.reset_index(drop=True)

## 3. Polynomial fit to weigh-in

* Many weigh-ins, but at irregular intervals
* Need to fit to this, which is easy
* Need to predict on regular date intervals, which is tricker

#### Get dates in a format that can be polynomial fitted

In [158]:
weight_df['timestamp'] = weight_df.Date.apply(lambda x: int(round(x.timestamp())))

#### Fit the polynomial function, get parameters

In [159]:
params = np.polyfit(weight_df['timestamp'], weight_df['Weight'], 3)

#### Get an effective x-range for the fit to display on

In [160]:
date_range = pd.date_range(start=weight_df.Date.min(),
                           end=weight_df.Date.max(),
                           freq='D', )
date_range_sec = [int(round(x.timestamp())) for x in date_range]

#### Get polynomial function value for each day in that range

In [161]:
poly_func = np.poly1d(params)
poly_weights = poly_func(date_range_sec)

#### Toss into a dataframe

In [162]:
poly_df = pd.DataFrame({'Date': date_range, 'Weight': poly_weights})

## 4. Start Plotting!

In [163]:
import time
from datetime import datetime as dt
from bokeh.models import (
    Select, LinearAxis, Range1d,
    LogColorMapper, HoverTool,
    Arrow, NormalHead, Label
)
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Plasma10
from bokeh.io import show

Plasma10.reverse()

In [166]:
# Where are we shipping this out to?
output_file("datetime.html")

# Create a new plot with a datetime axis type
p = figure(plot_width=1100,
           plot_height=500,
           x_axis_type="datetime",
           title="Run Log and Weigh-Ins",
           toolbar_location="above")

# Create a second y-axis, set y-axis ranges
p.extra_y_ranges = {"pace": Range1d(start=12, end=7.33)}
p.y_range = Range1d(170, 245)

# Adding the second axis to the plot.  
p.add_layout(LinearAxis(y_range_name="pace"), 'right')

# Label the axes
p.xaxis.axis_label = "Date"
p.yaxis[0].axis_label = "Weight (lbs)"
p.yaxis[1].axis_label = "Pace (Minutes per Mile)"

# Redundant, but it looks cool.
log_cmap = LogColorMapper(palette=Plasma10, low=7.5, high=11.5)

# Plot running logs
run_glyph = p.circle(
    x='Date',
    y='Pace_min',
    size='Display_Size',
    line_color='Line_Color',
    line_width=2,
    fill_color={'field': 'Pace_min', 'transform': log_cmap},
    alpha=0.8,
    y_range_name="pace",
    source=run_df,
    legend="Run Log"
)

# Add hover-over details for run log
tooltips = [
    ("Date", "@Date{%F}"),
    ("Distance", "@Distance{1.1} mi"),
    ("Pace", "@Pace min/mi"),
    ("Duration", "@Duration"),
    ("Name", "@Name"),
    ("Description", "@Description"),
    ("Tracker", "@Tracker"),
]
p.add_tools(
    HoverTool(
        tooltips=tooltips,
        renderers=[run_glyph],
        formatters={"Date": "datetime"}
    )
)

# Plot weigh-ins as small dots
p.circle(x='Date', y='Weight', size=2, color='gray',
         source=weight_df, alpha=0.7)

# Plot the polynomial fit
p.line(x='Date', y='Weight', source=poly_df, legend="Weight (Poly Fit)")

# Legend tuning
p.legend.location = 'bottom_left'
p.legend.background_fill_alpha = 0.5

###### Start Annotations

# Add arrow and label for when we got our house
new_house = time.mktime(dt(2018, 1, 15, 0, 0, 0).timetuple())*1000
p.add_layout(Arrow(end=NormalHead(size=10, fill_color="gray"),
                   line_width=3, line_color="#666666",
                   x_start=new_house, x_end=new_house,
                   y_start=180, y_end=171))
p.add_layout(Label(x=new_house, y=182,
                   text="New House", text_baseline="middle",
                   text_align="center", text_color="#666666"))

# Add arrow and label for when I started KC Half Marathon Training
kc_half_train = time.mktime(dt(2018, 5, 1, 0, 0, 0).timetuple())*1000
p.add_layout(Arrow(end=NormalHead(size=10, fill_color="gray"),
                   line_width=3, line_color="#666666",
                   x_start=kc_half_train, x_end=kc_half_train,
                   y_start=220, y_end=171))
p.add_layout(Label(x=kc_half_train, y=223,
                   text="Start KC Half Marathon Training",
                   text_baseline="middle", text_align="center",
                   text_color="#666666"))

# Add arrow and label for when I started KC Half Marathon Training
tendonitis_IF = time.mktime(dt(2019, 2, 1, 0, 0, 0).timetuple())*1000
p.add_layout(Arrow(end=NormalHead(size=10, fill_color="gray"),
                   line_width=3, line_color="#666666",
                   x_start=tendonitis_IF, x_end=tendonitis_IF,
                   y_start=179, y_end=171))
p.add_layout(Label(x=tendonitis_IF, y=184.5,
                   text="Achiles Tendonitis",
                   text_baseline="middle", text_align="center",
                   text_color="#666666"))
p.add_layout(Label(x=tendonitis_IF, y=181,
                   text="Started 16:8 IF",
                   text_baseline="middle", text_align="center",
                   text_color="#666666"))

# Ship it.
show(p)